# Lock\Slave

Calculate 2 axis gimble angles to point a targeting pod towards a target.
The inputs are the geo-coordinates and the elevation (MSL) of both aircraft and target.

In [12]:
import numpy as np
#from pymavlink import mavutil
lat1, lon1, elev1 = -35.3632621, 149.1652374, 584+10  # (From MAVLINK)
lat2, lon2, elev2 = -35.3564345, 149.1661153, 584     # can be an input or from designation

Using Haversine formula to calculate target distance, then calculate the bearing to form a 3d vector.

In [13]:
# Convert the GPS coordinates from degrees to radians
lat1_rad, lon1_rad = np.deg2rad(lat1), np.deg2rad(lon1)
lat2_rad, lon2_rad = np.deg2rad(lat2), np.deg2rad(lon2)


R = 6371000 # Earth radius in meters
dlat = lat2_rad - lat1_rad
dlon = lon2_rad - lon1_rad

# Calculate the distance between the two points with Haversine fourmula
a = np.sin(dlat/2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon/2)**2
c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
distance = R * c

# Calculate the bearing
bearing = np.arctan2(np.sin(dlon) * np.cos(lat2_rad),
                    np.cos(lat1_rad) * np.sin(lat2_rad) -
                    np.sin(lat1_rad) * np.cos(lat2_rad) * np.cos(dlon))

# Calculate the x, y and z distances between the two points
x_distance = distance * np.cos(bearing)
y_distance = distance * np.sin(bearing)
z_distance = elev1 - elev2

Converting the targeting vector from earth (NED) reference frame to aircraft (Body) reference frame

In [14]:
v_ned = np.array([x_distance, y_distance, z_distance]) 

# Define the rotation matrix that describes the orientation of the body (From MAVLINK)
phi = np.deg2rad(15) # Rotation around x-axis in radians 
theta = np.deg2rad(20) # Rotation around y-axis in radians
psi = np.deg2rad(30) # Rotation around z-axis in radians

R_bn = np.array([
[np.cos(theta)*np.cos(psi), np.cos(theta)*np.sin(psi), -np.sin(theta)],
[np.sin(phi)*np.sin(theta)*np.cos(psi)-np.cos(phi)*np.sin(psi), np.sin(phi)*np.sin(theta)*np.sin(psi)+np.cos(phi)*np.cos(psi), np.sin(phi)*np.cos(theta)],
[np.cos(phi)*np.sin(theta)*np.cos(psi)+np.sin(phi)*np.sin(psi), np.cos(phi)*np.sin(theta)*np.sin(psi)-np.sin(phi)*np.cos(psi), np.cos(phi)*np.cos(theta)]
])
# Transform the vector v from the NED frame to the body frame
v_body = np.matmul(R_bn, v_ned)

Get azimuth angle φ and polar angle θ of the vector in which the gimble will adjust its motors in order to point at the target location.

In [15]:
# Define the vector v
v = v_body # 3 x 1 matrix

# Compute the direction angles with respect to each axis
Azimuth = np.arctan2(v[1], v[0]) # Azimuth angle
Polar = np.arctan2(v[2], np.sqrt(v[0]**2 + v[1]**2)) # Polar angle

# Convert the angles to degrees

print(f'Azimuth:  {np.rad2deg(Azimuth)}')
print(f'Polar:  {np.rad2deg(Polar)}')
print(f'distance: {np.round(distance,3)}')
print(f'bearing: {np.round(np.rad2deg(bearing),3)}')
print(v_body)

Azimuth:  -19.896327814936402
Polar:  24.76857968151537
distance: 763.357
bearing: 5.987
[ 651.81663006 -235.90707681  319.83912761]


Negative Polar angles for pointing upward.

Positive Azimuth angles for pointing to the right.

0 Azimuth angles for pointing forward.

# Designation

Designate a target area and calcuate bearing and distance, coordinates can be fingure out from bearing and distance again using Haversine fourmula

In [16]:
def calculate_coordinates(lat1, lon1, distance, bearing):
    # Convert input values from degrees to radians
    lat1_rad = np.deg2rad(lat1)
    lon1_rad = np.deg2rad(lon1)
    bearing_rad = (bearing)

    # Earth radius in kilometers
    radius = 6371000

    # Calculate the angular distance in radians
    angular_distance = distance / radius

    # Calculate the latitude of the destination point using the haversine formula
    lat2_rad = np.arcsin(np.sin(lat1_rad) * np.cos(angular_distance) +
                         np.cos(lat1_rad) * np.sin(angular_distance) * np.cos(bearing_rad))

    # Calculate the longitude of the destination point using the haversine formula
    lon2_rad = lon1_rad + np.arctan2(np.sin(bearing_rad) * np.sin(angular_distance) * np.cos(lat1_rad),
                                      np.cos(angular_distance) - np.sin(lat1_rad) * np.sin(lat2_rad))

    # Convert the latitude and longitude from radians to degrees
    lat2 = np.degrees(lat2_rad)
    lon2 = np.degrees(lon2_rad)

    return lat2, lon2

Azimuth and elevation are the tilt and pan of the of targeting pod gimbal, using same tilt and pan from the lock/slave to varify the coordinates

In [17]:
# Define the azimuth and elevation angles
azimuth = (Azimuth)   # Example azimuth angle in radians
elevation = (Polar)   # Example elevation angle in radians

# Calculate the components of the unit vector
x = np.cos(azimuth) * np.cos(elevation)
y = np.sin(azimuth) * np.cos(elevation)
z = np.sin(elevation)

# Create the unit vector
unit_vector = np.array([x, y, z])

# Print the unit vector
print("Unit vector: ", unit_vector)

Unit vector:  [ 0.85380835 -0.30901242  0.41895421]


By using the lidar or laser to calulate the distance between aircraft and target.

In [18]:
# Define the magnitude of the vector
magnitude = np.sqrt(x_distance**2 + y_distance**2 + z_distance**2)

# Multiply the magnitude by the unit vector
v_scaled = magnitude * unit_vector

# Print the scaled vector
print("Scaled vector v': ", v_scaled)

# Transform the vector v from the body frame to the NED frame
v_ned = np.matmul(R_bn.T, v_scaled)
elev2 = elev1 - v_ned[2]

print(f'vector in earth frame: {v_ned}')

Scaled vector v':  [ 651.81663006 -235.90707681  319.83912761]
vector in earth frame: [759.19412817  79.61414076  10.        ]


In [19]:
n_bearing = np.arctan2(v_ned[1], v_ned[0]) # Azimuth angle
elevation = np.arctan2(v_ned[2], np.sqrt(v_ned[0]**2 + v_ned[1]**2)) # Polar angle
n_distance = magnitude * np.cos(elevation)

print(f'elevation:  {np.round(np.rad2deg(elevation),5)}')
print(f'bearing:  {np.round(np.rad2deg(n_bearing),5)}')
print(f'distance:  {n_distance}')

elevation:  0.75053
bearing:  5.98654
distance:  763.3571481702102


In [20]:
print(calculate_coordinates(lat1,lon1,n_distance,n_bearing))
print(elev2)

(-35.3564345, 149.1661153)
584.0


# -35.3564345, 149.1661153, 584
Ended up with the same coordinates by using gimbal angles (tilt, pan)

Transfrom the coordinates format from decimal to DMS:

In [53]:
def decimal_degrees_to_dms(Decimal,lat:bool = False, lon:bool = False):
    
    d = int(Decimal)
    m = int((Decimal - d) * 60)
    s = round((Decimal - d - m/60) * 3600.00,2)

    if lat == True:
        if d < 0:
            H = 'S'
        else:
            H = 'N'

    if lon == True:
        if d < 0:
            H = 'W'
        else:
            H = 'E'
    
    return (f"{abs(d)}°{abs(m)}'{abs(s)}"'"'f"{H}")

In [54]:
lat_DMS = decimal_degrees_to_dms(lat1, lat=True)
lon_DMS = decimal_degrees_to_dms(lon1, lon=True)
print(lat_DMS,lon_DMS)

35°21'47.74"S 149°9'54.85"E
